# Construyendo modelos con PyTorch

## Modelos básicos

### `torch.nn.Module` y `torch.nn.Parameter`


`torch.nn.Module` es la clase base para cualquier red neuronal en PyTorch. Cualquier clase que herede de `torch.nn.Module` debe implementar el método `forward`. El método `forward` es el que define cómo se calcula la salida de la red neuronal.

Cualquier objeto de tipo `torch.nn.Module` registra todos los parámetros de la red neuronal (los pesos y los sesgos). Estos parámetros son objetos de tipo `torch.nn.Parameter`, que es una subclase de `torch.Tensor`. Los parámetros se pueden acceder a través del método `parameters()` de la clase `Module`.




A continuación se muestra un ejemplo de cómo definir una red neuronal muy básica en PyTorch. Esta red consta de dos capas lineales y una función de activación ReLU entre ellas. En ella podemos ver la estructura básica de una red neuronal en PyTorch, con un método `__init__()` que define las capas y otros componentes de la red, y un método `forward()` donde se realiza la computación.

In [2]:
import torch

class TinyModel(torch.nn.Module):
    
    def __init__(self): # Definimos las capas como atributos 
        super().__init__()
        self.linear1 = torch.nn.Linear(100, 200) # Capa de entrada
        self.activation = torch.nn.ReLU() # Función de activación
        self.linear2 = torch.nn.Linear(200, 10) # Capa de salida
        self.softmax = torch.nn.Softmax() # Función de salida
    
    def forward(self, x): # Definimos el flujo de datos
        x = self.linear1(x) # Capa de entrada
        x = self.activation(x) # Función de activación
        x = self.linear2(x) # Capa de salida
        x = self.softmax(x) # Función de salida
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0165, -0.0320, -0.0229,  ..., -0.0139, -0.0368,  0.0666],
        [-0.0647,  0.0893, -0.0493,  ...,  0.0397, -0.0546, -0.0781],
        [-0.0740, -0.0500, -0.0597,  ...,  0.0890, -0.0705, -0.0604],
        ...,
        [-0.0829, -0.0146,  0.0345,  ..., -0.0057,  0.0829,  0.0737],
        [-0.0223, -0.0434, -0.0292,  ..., -0.0340,  0.0451,  0.0523],
        [-0.0747, -0.0579,  0.0683,  ...,  0.0150, -0.0853,  0.0460]],
       requires_grad=True)
Parameter containing:
tensor([-0.0587, -0.0109,  0.0101, -0.0069,  0.0317,  0.0026,  0.0690, -0.0775,
        -0.0985,  0.0605, -0.0362,  0.0442,  0.0301, -0.0218, -0.0363,  0.0097,
         0.0359,  0.0279, -0.0797, -0.03

## Estilo funcional

La librería [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html) permite llamar a algunos de los elementos (típicamente funciones de activación) directamente como funciones en lugar de cómo atributos de un objeto. Por ejemplo, el modelo anterior es equivalente al siguiente.

In [ ]:
class TinyModel(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(100, 200) # Capa de entrada
        self.activation = torch.nn.ReLU() # Función de activación
        self.linear2 = torch.nn.Linear(200, 10) # Capa de salida
        self.softmax = torch.nn.Softmax() # Función de salida
    
    def forward(self, x):
        x = self.linear1(x) 
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

In [9]:
import torch
import torch.nn.functional as F 

class TinyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(100, 200)
        self.linear2 = torch.nn.Linear(200, 10) 

    def forward(self, x):
        x = self.linear1(x) 
        x = F.relu(x)
        x = self.linear2(x)
        x = F.softmax(x)
        return x
    
    # o también es equivalente a:
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.softmax(self.linear2(x))
        return x
    
    # o aún más compacto:
    def forward(self, x):
        return F.softmax(self.linear2(F.relu(self.linear1(x))))

### Usando `Secuential`

`nn.Sequential` es una clase que permite definir una red neuronal secuencialmente. Es decir, se pueden definir las capas de la red neuronal en el orden en el que se van a aplicar. A continuación se muestra cómo se puede definir el modelo anterior usando `nn.Sequential`.

In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential( # Definimos las capas en orden como un único atributo 
        nn.Linear(100, 200),
        nn.ReLU(),
        nn.Linear(200, 10),
        nn.Softmax()
    )

  def forward(self, x):
    x = self.model(x) # Se llama a toda la secuencia, su orden ya está definido internamente
    return x


## Capas lineales

El tipo más básico de capa de red neuronal es una capa *lineal* o *totalmente conectada*. Esta es una capa en la que cada entrada influye en cada salida de la capa en un grado especificado por los pesos de la capa. Si un modelo tiene *m* entradas y *n* salidas, los pesos serán una matriz *m* x *n*.

Se llama *lineal* porque la salida de la capa es una combinación lineal de las entradas $y=Wx+b$, donde $W$ es la matriz de pesos, $x$ es el vector de entradas y $b$ es el vector de sesgos.

Si tenemos 3 entradas $x_1$, $x_2$ y $x_3$ y 2 salidas $y_1$ y $y_2$, la salida de la capa será:

$$\begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23} \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}$$

In [17]:
lin = torch.nn.Linear(3, 2) # 3 entradas, 2 salidas

print('Parámetros de la capa (pesos y sesgos):')
for param in lin.parameters():
    print(param)

# Lo mismo accediendo directamente a cada atributo:
print('\n\nPesos:', lin.weight)
print('\n\nSesgos:', lin.bias)

x = torch.rand(1, 3) # Tensor de entrada de 1x3
print('\n\nInput:', x)

y = lin(x)
print('\n\nOutput:')
print(y)

Parámetros de la capa (pesos y sesgos):
Parameter containing:
tensor([[-0.2069,  0.2911, -0.0554],
        [-0.3530, -0.0932,  0.4839]], requires_grad=True)
Parameter containing:
tensor([-0.5717, -0.4867], requires_grad=True)


Pesos: Parameter containing:
tensor([[-0.2069,  0.2911, -0.0554],
        [-0.3530, -0.0932,  0.4839]], requires_grad=True)


Sesgos: Parameter containing:
tensor([-0.5717, -0.4867], requires_grad=True)


Input: tensor([[0.4343, 0.9277, 0.8893]])


Output:
tensor([[-0.4408, -0.2962]], grad_fn=<AddmmBackward0>)


Podemos ver que `lin.weight` contiene la matriz de pesos y que `lin.bias` contiene el vector de sesgos, siendo ambos de tipo `Parameter`.

`Parameter` es una subclase de `Tensor` que se utiliza para indicar que un tensor es un parámetro de una red neuronal y, por lo tanto, debe registrar los gradientes por el módulo de autograd de PyTorch. Esto es importante para que PyTorch pueda calcular los gradientes de los parámetros durante el entrenamiento.

## Fuentes

- https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html